In [14]:
from test import EigenData
import numpy as np
import scipy.io
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
%matplotlib inline

In [15]:
data = EigenData()
eigenworms = data.get_eigenworms('EigenWorms.mat')

footage = data.get_footage('20150814-All-PNAS2011-DataStitched .mat')
newdata = torch.rand(33600,5)
for k in footage.keys():
    print(k, footage[k].shape)
    arr1=footage[k]

    arr2=arr1[:5,]

    r = data.reconstruct(footage[k])
    print('-->', r.shape)
    dataset1 = arr2.transpose()

    newdata = np.concatenate((newdata,dataset1),axis=0)
newdata = newdata[33600:,]
print(newdata.shape)

b (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
c (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
d (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
e (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
f (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
g (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
h (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
i (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
j (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
k (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
l (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
m (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
(403200, 5)


In [16]:
first_5_eigenworms = eigenworms[0:5,:]
eig_worm_0 = eigenworms[0,:]
eig_worm_1 = eigenworms[1,:]
eig_worm_2 = eigenworms[2,:]
eig_worm_3 = eigenworms[3,:]
eig_worm_4 = eigenworms[4,:]

In [17]:
data1=newdata[~np.isnan(newdata).any(axis=1)]

data1 = data1.astype('float32')
print(data1.shape)
print(len(data1))

(386422, 5)
386422


In [18]:
def create_dataset(data,n_predictions,n_next):
    
   
    dim = data.shape[1]
    train_X, train_Y = [], []
    for i in range(data.shape[0]-n_predictions-n_next-1):
        a = data[i:(i+n_predictions),:]
        train_X.append(a)
        tempb = data[(i+n_predictions):(i+n_predictions+n_next),:]
        b = []
        for j in range(len(tempb)):
            for k in range(dim):
                b.append(tempb[j,k])
        train_Y.append(b)
    train_X = np.array(train_X,dtype='float64')
    train_Y = np.array(train_Y,dtype='float64')

    return train_X, train_Y

In [19]:
X, Y = create_dataset(data1,3,2)

print(X.shape)
print(Y.shape)

(386416, 3, 5)
(386416, 10)


In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))
data1 = scaler.fit_transform(data1)
Y = scaler.fit_transform(Y)

In [21]:
data_X, data_Y = create_dataset(data1,3,2)

print(data_X.shape)
print(data_Y.shape)

(386416, 3, 5)
(386416, 10)


In [22]:
train_size = int(len(data_X) * 0.8)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = Y[:train_size]
test_X = data_X[train_size:]
test_Y = Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(309132, 3, 5)
(309132, 10)
(77284, 3, 5)
(77284, 10)


In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input,LSTM,Dropout,Dense


timesteps = 3
# create and fit the LSTM network

model = keras.Sequential()
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(10,activation='linear'))
model.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(train_X, train_Y, epochs=100, batch_size=512, verbose=2, validation_data=(test_X, test_Y))

Epoch 1/100
604/604 - 19s - loss: 0.0102 - root_mean_squared_error: 0.1008 - val_loss: 0.0026 - val_root_mean_squared_error: 0.0509
Epoch 2/100
604/604 - 14s - loss: 0.0039 - root_mean_squared_error: 0.0622 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0385
Epoch 3/100
604/604 - 14s - loss: 0.0024 - root_mean_squared_error: 0.0490 - val_loss: 7.8976e-04 - val_root_mean_squared_error: 0.0281
Epoch 4/100
604/604 - 14s - loss: 0.0016 - root_mean_squared_error: 0.0399 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0325
Epoch 5/100
604/604 - 14s - loss: 0.0012 - root_mean_squared_error: 0.0345 - val_loss: 6.2830e-04 - val_root_mean_squared_error: 0.0251
Epoch 6/100
604/604 - 14s - loss: 9.6713e-04 - root_mean_squared_error: 0.0311 - val_loss: 2.7537e-04 - val_root_mean_squared_error: 0.0166
Epoch 7/100
604/604 - 15s - loss: 8.1437e-04 - root_mean_squared_error: 0.0285 - val_loss: 3.9841e-04 - val_root_mean_squared_error: 0.0200
Epoch 8/100
604/604 - 14s - loss: 7.1017e-04 - root_

Epoch 60/100
604/604 - 13s - loss: 4.1190e-04 - root_mean_squared_error: 0.0203 - val_loss: 1.0710e-04 - val_root_mean_squared_error: 0.0103
Epoch 61/100
604/604 - 13s - loss: 4.1052e-04 - root_mean_squared_error: 0.0203 - val_loss: 1.2288e-04 - val_root_mean_squared_error: 0.0111
Epoch 62/100
604/604 - 14s - loss: 4.0948e-04 - root_mean_squared_error: 0.0202 - val_loss: 1.1121e-04 - val_root_mean_squared_error: 0.0105
Epoch 63/100
604/604 - 14s - loss: 4.0803e-04 - root_mean_squared_error: 0.0202 - val_loss: 1.0253e-04 - val_root_mean_squared_error: 0.0101
Epoch 64/100
604/604 - 14s - loss: 4.0681e-04 - root_mean_squared_error: 0.0202 - val_loss: 1.1476e-04 - val_root_mean_squared_error: 0.0107
Epoch 65/100
604/604 - 14s - loss: 4.0508e-04 - root_mean_squared_error: 0.0201 - val_loss: 9.3537e-05 - val_root_mean_squared_error: 0.0097
Epoch 66/100
604/604 - 14s - loss: 4.0544e-04 - root_mean_squared_error: 0.0201 - val_loss: 1.1935e-04 - val_root_mean_squared_error: 0.0109
Epoch 67/100


In [24]:
testPredict = model.predict(test_X)

In [25]:
newtestPredict = scaler.inverse_transform(testPredict)
newtest_Y = scaler.inverse_transform(test_Y)

In [26]:
testScore = sqrt(mean_squared_error(newtest_Y, newtestPredict))
print('Test Score: %.5f RMSE' % (testScore))

Test Score: 0.30611 RMSE
